# **Run hyper parameter scan on SVHN (cropped)**

First, install kerastuner and import the neccesary libraries


In [1]:
!pip install -U keras-tuner
!export KERASTUNER_TUNER_ID="tuner0"
!export KERASTUNER_ORACLE_IP="127.0.0.1"
!export KERASTUNER_ORACLE_PORT="8000"
import os
import IPython
import kerastuner as kt

%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)
import tensorflow_datasets as tfds
AUTO = tf.data.experimental.AUTOTUNE

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.1)
Tensorflow version 2.2.0


Set up *TPU* accelerators (if available)

In [2]:
try: # detect TPUs
  tpu = None
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError: # detect GPUs
  strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
  #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Initializing the TPU system: grpc://10.106.242.42:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.106.242.42:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Number of accelerators:  8


Authenticate

In [3]:
from google.colab import auth
auth.authenticate_user()

Get the SVHN cropped dataset (32,32,3) streetnumber images (! Remember to create your own bucket the Cloud can access and pass it to data_dir)

In [4]:
if tpu:
  BATCH_SIZE = 16*strategy.num_replicas_in_sync  # A TPU has 8 cores so this will be 128
else:
  BATCH_SIZE = 32  # On Colab/GPU, a higher batch size does not help and sometimes does not fit on the GPU (OOM)

def convert_dataset(item):
    """Puts the mnist dataset in the format Keras expects, (features, labels)."""
    image = item['image']
    label = item['label']
    label = tf.one_hot(tf.squeeze(label), 10)
    image = tf.dtypes.cast(image, 'float32') / 255.
    return image, label
    
ds_data = tfds.load('svhn_cropped',data_dir="gs://thaarres_bucket")#'gs://tfds-data/datasets')
ds_train, ds_test = ds_data['train'], ds_data['test']
ds_train = ds_train.map(convert_dataset)
ds_train = ds_train.cache()
ds_train = ds_train.repeat()
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTO)
ds_test = ds_test.map(convert_dataset)
ds_test = ds_test.cache()
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(AUTO)



for img_feature, label in ds_train:
      break
print(" --------------INPUT INFO --------------")
print('INPUT img_feature.shape (batch_size, image_height, image_width) =', img_feature.shape)
print('INPUT label.shape (batch_size, number_of_labels) =', label.shape)
IMAGE_SIZE = [img_feature.shape[1], img_feature.shape[2]]

EPOCHS = 20
steps_per_epoch=73257//BATCH_SIZE


 --------------INPUT INFO --------------
INPUT img_feature.shape (batch_size, image_height, image_width) = (128, 32, 32, 3)
INPUT label.shape (batch_size, number_of_labels) = (128, 10)


Do a small training to check that everything is working correctly

In [5]:
# #Do test
# with strategy.scope(): # this line is all that is needed to run on TPU (or multi-GPU, ...)

#   model = tf.keras.Sequential([
#       tf.keras.layers.Conv2D(kernel_size=3, filters=16, padding='same', activation='relu', input_shape=[*IMAGE_SIZE, 3]),
#       tf.keras.layers.Conv2D(kernel_size=3, filters=30, padding='same', activation='relu'),
#       tf.keras.layers.MaxPooling2D(pool_size=2),
#       tf.keras.layers.Conv2D(kernel_size=3, filters=60, padding='same', activation='relu'),
#       tf.keras.layers.MaxPooling2D(pool_size=2),
#       tf.keras.layers.Conv2D(kernel_size=3, filters=90, padding='same', activation='relu'),
#       tf.keras.layers.MaxPooling2D(pool_size=2),
#       tf.keras.layers.Conv2D(kernel_size=3, filters=110, padding='same', activation='relu'),
#       tf.keras.layers.MaxPooling2D(pool_size=2),
#       tf.keras.layers.Conv2D(kernel_size=3, filters=130, padding='same', activation='relu'),
#       tf.keras.layers.Conv2D(kernel_size=1, filters=40, padding='same', activation='relu'),
#       tf.keras.layers.GlobalAveragePooling2D(),
#       tf.keras.layers.Dense(10, activation='softmax')
#   ])

#   model.compile(
#     optimizer='adam',
#     loss= 'categorical_crossentropy',
#     metrics=['accuracy'])

#   model.summary()

#   history = model.fit(ds_train, steps_per_epoch=steps_per_epoch, epochs=5,
#                     validation_data=ds_test)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 16)        448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 30)        4350      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 30)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 60)        16260     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 60)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 90)          48690     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 90)          0

Then define the model

In [16]:
def build_model(hp):
    inputs = tf.keras.Input(shape=[*IMAGE_SIZE, 3])
    x = inputs
    for i in range(hp.Int('conv_layers', 1, 3, default=3)):
        x = tf.keras.layers.Conv2D( #tf.keras.layers.SeparableConv2D(
            filters=hp.Int('filters_' + str(i), 4, 32, step=4, default=8),
            kernel_size=hp.Int('kernel_size_' + str(i), 3, 5),
            activation='relu',
            padding='same')(x)

        if hp.Choice('pooling' + str(i), ['max', 'avg']) == 'max':
            x = tf.keras.layers.MaxPooling2D()(x)
        else:
            x = tf.keras.layers.AveragePooling2D()(x)

        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)

    if hp.Choice('global_flatten', ['max', 'avg']) == 'max':
        x = tf.keras.layers.GlobalMaxPooling2D()(x)
    else:
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
    # else:
    #   x = tf.keras.layers.Flatten()
    for i in range(hp.Int('dense_layers', 0, 2, default=1)):
        x = tf.keras.layers.Dense(
            units=hp.Int('neurons_' + str(i), 64, 256, step=64, default=128),
            activation='relu')(x)

    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)

    # optimizer = hp.Choice('optimizer', ['adam', 'sgd'])
    lr = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log', default=1e-3)
    model.compile(tf.keras.optimizers.Adam(lr), loss='categorical_crossentropy', metrics=['accuracy'])
    print("Testing model:")
    model.summary()
    return model

Define the keras-tuner object to run the hyperparameter search

In [18]:
# !gsutil rm -r gs://bucket/subdir
tuner = kt.Hyperband(
        hypermodel=build_model,
        objective='val_accuracy',
        max_epochs=100,
        factor=3,
        hyperband_iterations=3,
        distribution_strategy=strategy,
        directory='gs://thaarres_bucket/keras-tune-svhn',
        project_name='v1_conv2D_globPool',
        overwrite=True)

Testing model:
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 8)         224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 8)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 16, 8)         32        
_________________________________________________________________
re_lu (ReLU)                 (None, 16, 16, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 8)         584       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 8)  

Before running the search, define a callback to clear  training outputs after every training step.

In [19]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

Run scan

In [ ]:
 EPOCHS=100
 
 print(" Steps per epoch is {}".format(steps_per_epoch))
 tuner.search(ds_train,
                 steps_per_epoch=steps_per_epoch,
                 validation_data=ds_test,
                 epochs=EPOCHS,
                 callbacks=[tf.keras.callbacks.EarlyStopping('val_accuracy'),ClearTrainingOutput()])

Testing model:
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 24)        672       
_________________________________________________________________
average_pooling2d (AveragePo (None, 16, 16, 24)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 16, 24)        96        
_________________________________________________________________
re_lu (ReLU)                 (None, 16, 16, 24)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 8)         3080      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 8)  

Get the optimal hyperparameters

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of conv layers is {best_hps.get('conv_layers')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Then retrain using the best model!

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(ds_train, epochs = 10, validation_data = ds_test)